In [ ]:
import requests
# from sql_db.visit import Visit

SERVER_URL = "http://localhost:8000"


def get_visits():
    r = requests.get(f"{SERVER_URL}/all_visits")
    answ = r.json()
    return answ


In [ ]:
visits = get_visits()
print(f"visits: {visits}")

In [ ]:
len(visits)

In [ ]:
visits[0].keys()

In [ ]:
import faiss
import numpy as np
from openai import OpenAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_embedding(text):
    response = client.embeddings.create(
    model="text-embedding-3-small",
    input=text
    )

    return np.array(response.data[0].embedding, dtype=np.float32)

In [ ]:
from tqdm import tqdm
embedding_list = []
id_to_visit = []

# input:
# "interview_description": "opis wizyty...",
# "treatment_description": "badanie...",
# output:
# "applied_medicines"
#  {visit['recommendation']}

def get_visit_text(visit):
    visit_text = f"{visit['interview_description']} {visit['treatment_description']}"
    return visit_text

for visit in tqdm(visits):
    visit_text = get_visit_text(visit) 
    emb = get_embedding(visit_text)
    embedding_list.append(emb)
    id_to_visit.append(visit['id_visit'])

In [ ]:
embeddings_array = np.vstack(embedding_list)
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

print(f"Added {len(embeddings_array)} vectors.")

In [ ]:
index_file_name = "vet_visits.index"
faiss.write_index(index, index_file_name)
print(f"Saved in '{index_file_name}'")

In [ ]:
import pickle
id_file_name = "id_to_visit.pkl"
with open(id_file_name, "wb") as f:
    pickle.dump(id_to_visit, f)

# Read

In [ ]:
loaded_index = faiss.read_index(index_file_name)
with open(id_file_name, "rb") as f:
    loaded_id_to_visit = pickle.load(f)

In [ ]:
from pypdf import PdfReader

PDF_PATH = "../data/Walker.pdf"


def read_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

In [ ]:
pdf_text = read_pdf(PDF_PATH)
pdf_text

test_dict = {
    "interview_description": "Dziś pierwszy raz napady drgawkowe z niecałkowitą utratą przytomności, pierwszy o godzinie 18:20. W trakcie ataku pies otwierał szeroko oczy i głęboko oddychał, pomiędzy atakami całkowita utrata przytomności, nie oddał ani kału ani moczu. Każdy z ataków trwał około 30-40 sekund. Do godziny 21:00 około 7 ataków. Ostatni posiłek jadł rano, ma stały dostęp do karmy. Pies nie ma dostępu do trucizn ani nawozów, mieszka w kojcu, dziś rano jadł standardowe jedzenie. Około miesiąc temu przebył babeszjozę potwierdzoną w gabinecie weterynaryjnym, otrzymał dwie kroplówki dożylne, nie kontynuowano leczenia. Wcześniej nie wykazywał niepokojących objawów. Jest psem adoptowanym, właściciele nie znają jego wcześniejszych losów, w schronisku nie miał ataków, wcześniej silne zapalenie płuc. Zaszczepiony przeciwko wściekliźnie i chorobom wirusowym. Pies jest agresywny. Dziś przywieziony nieprzytomny, mokry, właścicielka schładzała polewając wodą.",
    "treatment_description": "Podłączono psa do maski tlenowej, założono wkłucie dożylne, pobrano krew na szeroki profil z jonogramem, podano Płyn Ringera 600 ml w tempie 400 ml/h, w trakcie ataku podano propofol 4,5 ml i.v., następnie dodatkowo 1 ml propofolu i.v. Odbyły się badania laboratoryjne (krew, T4, fT4), monitorowano parametry życiowe. Przygotowano psa do wyjazdu do lecznicy całodobowej na monitoring.",
    "applied_medicines": "Propofol-Lipuro 10mg/1ml 5,5 ml, Płyn Ringera 600 ml",
    "recommendation": "Nocny monitoring w lecznicy całodobowej, diagnostyka w celu wykluczenia zaburzeń kardiologicznych oraz pełne badanie neurologiczne po odzyskaniu przez psa przytomności. Oczekiwanie na wynik oznaczenia hormonów tarczycy. Kontakt w tej sprawie z lecznicą."
}

In [ ]:
def find_in_vector_db(interview_description, treatment_description, k):
    # Tworzymy embedding dla zapytania
    query_text = f"{interview_description}  {treatment_description}"
    query_vector = get_embedding(query_text).reshape(1, -1)

    # Wyszukujemy najbliższe przypadki w bazie FAISS
    distances, indices = loaded_index.search(query_vector, k=k)
    return distances, indices
    

In [ ]:
interview_description = test_dict["interview_description"]
treatment_description = test_dict["treatment_description"]
k = 3

In [ ]:
distances, indices = find_in_vector_db(interview_description, treatment_description, k)
distances, indices

In [ ]:
def get_similar_visits_info(indices, visits):
    # Zbieramy leki i zalecenia z najbliższych wizyt
    similar_visits_info = []
    for idx in indices[0]:
        visit_id = loaded_id_to_visit[idx]
        visit = next(v for v in visits if v['id_visit'] == visit_id)
        similar_visits_info.append({
            "applied_medicines": visit['applied_medicines'],
            "recommendation": visit['recommendation']
        })
    return similar_visits_info

In [ ]:
similar_visits_info = get_similar_visits_info(indices, visits)
similar_visits_info

In [ ]:
similar_visits_info = []
for idx in indices[0]:
    visit_id = loaded_id_to_visit[idx]
    visit = next(v for v in visits if v['id_visit'] == visit_id)
    print(idx, visit_id, visit['applied_medicines'], visit['recommendation'])
    similar_visits_info.append({
        "applied_medicines": visit['applied_medicines'],
        "recommendation": visit['recommendation']
    })

In [ ]:

def generate_prompt_for_new_visit(interview_description, treatment_description, k=3):
    """
    Generuje prompt dla nowej wizyty na podstawie podobnych przypadków w bazie.

    ```
    Parametry:
    - interview_description: tekst wywiadu pacjenta
    - treatment_description: opis leczenia
    - k: liczba najbliższych przypadków do uwzględnienia
    """
    distances, indices = find_in_vector_db(interview_description, treatment_description, k)

    similar_visits_info = get_similar_visits_info(indices, visits)


    # Tworzymy prompt dla LLM z informacjami o podobnych przypadkach
    prompt = f"Nowa wizyta:\nWywiad: {interview_description}\nLeczenie: {treatment_description}\n\nPodobne przypadki i ich zalecenia:\n"
    for i, info in enumerate(similar_visits_info, 1):
        prompt += f"{i}. Leki: {info['applied_medicines']}, Zalecenia: {info['recommendation']}\n"

    prompt += "\nNa podstawie tych przykładów, wygeneruj zalecenia i przepisz leki dla nowej wizyty."

    return prompt

In [ ]:
prompt = generate_prompt_for_new_visit(interview_description, treatment_description, k)
print(prompt)

In [ ]:
test_dict["treatment_description"]

In [ ]:
query_vector = embeddings[0:1]


In [ ]:
distances, indices = loaded_index.search(query_vector, k=3)

for idx, dist in zip(indices[0], distances[0]):
    print(f"id_visit: {loaded_id_to_visit[idx]}, distance: {dist}")